In [1]:
# =============================================================================
# QB DECISION TRAINER - STREAMLINED NOTEBOOK
# =============================================================================
# This notebook runs the coaching app using pre-exported data files.
# Required files in DATA_PATH:
#   - receivers_app.parquet
#   - plays_app.parquet
#   - tracking_app.parquet
#   - supplementary_app.parquet
# =============================================================================

# %% [markdown]
# # Cell 1: Imports

# %%
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, callback, Output, Input, State, no_update
import plotly.graph_objects as go

In [2]:
# =============================================================================
# Cell 2: Load Data
# =============================================================================

# Paths
DATA_PATH = r'C:\Users\carso\Downloads\nfl-big-data-bowl-2026-analytics'
SOURCE_PATH = rf'{DATA_PATH}\114239_nfl_competition_files_published_analytics_final\train'

print("Loading data files...")
print("=" * 70)

# Load app data files
receivers_app = pd.read_parquet(f'{DATA_PATH}/receivers_app.parquet')
plays_app = pd.read_parquet(f'{DATA_PATH}/plays_app.parquet')
tracking_app = pd.read_parquet(f'{DATA_PATH}/tracking_app.parquet')
supplementary_app = pd.read_parquet(f'{DATA_PATH}/supplementary_app.parquet')

print(f"✓ receivers_app: {len(receivers_app):,} rows")
print(f"✓ plays_app: {len(plays_app):,} rows")
print(f"✓ tracking_app: {len(tracking_app):,} rows")
print(f"✓ supplementary_app: {len(supplementary_app):,} rows")

# Split tracking into input/output
input_data = tracking_app[tracking_app['source'] == 'input'].copy()
output_data = tracking_app[tracking_app['source'] == 'output'].copy()

print(f"\n  Input tracking frames: {len(input_data):,}")
print(f"  Output tracking frames: {len(output_data):,}")

# -----------------------------------------------------------------------------
# Extract QB names from source files
# -----------------------------------------------------------------------------
print("\nExtracting QB names from source files...")

qb_frames = []
source_files = [f'input_2023_w{w:02d}.csv' for w in range(1, 19)] + \
               [f'input_2024_w{w:02d}.csv' for w in range(1, 19)]

for filename in source_files:
    filepath = f'{SOURCE_PATH}/{filename}'
    try:
        df = pd.read_csv(filepath, usecols=['game_id', 'play_id', 'player_role', 'player_name'])
        passers = df[df['player_role'] == 'Passer'][['game_id', 'play_id', 'player_name']].drop_duplicates()
        qb_frames.append(passers)
    except FileNotFoundError:
        continue

qb_lookup = pd.concat(qb_frames, ignore_index=True).drop_duplicates()
qb_lookup = qb_lookup.rename(columns={'player_name': 'qb_name'})

# Merge QB names into plays_app
plays_app = plays_app.merge(qb_lookup, on=['game_id', 'play_id'], how='left')
plays_app['qb_name'] = plays_app['qb_name'].fillna('QB')

print(f"✓ Added qb_name to plays_app ({plays_app['qb_name'].notna().sum():,} matched)")
print(f"  Unique QBs: {plays_app['qb_name'].nunique()}")

Loading data files...
✓ receivers_app: 63,219 rows
✓ plays_app: 13,770 rows
✓ tracking_app: 5,295,049 rows
✓ supplementary_app: 18,009 rows

  Input tracking frames: 4,754,317
  Output tracking frames: 540,732

Extracting QB names from source files...
✓ Added qb_name to plays_app (13,770 matched)
  Unique QBs: 80


In [3]:
# =============================================================================
# QB DECISION TRAINER - CONSOLIDATED APP
# =============================================================================
# Required source files:
#   - receivers_app.parquet
#   - plays_app.parquet
#   - tracking_app.parquet
#   - supplementary_app.parquet
#   - Source input CSVs for QB name extraction
# =============================================================================

# =============================================================================
# CELL 3: PLAY LOADER FUNCTION
# =============================================================================

def load_play_for_coaching(game_id, play_id):
    """Load all data needed for a single play in the coaching app."""
    preds = receivers_app[
        (receivers_app['game_id'] == game_id) & 
        (receivers_app['play_id'] == play_id)
    ].copy()
    
    if len(preds) == 0:
        return None
    
    play_info = plays_app[
        (plays_app['game_id'] == game_id) & 
        (plays_app['play_id'] == play_id)
    ]
    
    if len(play_info) == 0:
        return None
    
    play_info = play_info.iloc[0]
    
    # Get supplementary data for field markings and scores
    supp = supplementary_app[
        (supplementary_app['game_id'] == game_id) & 
        (supplementary_app['play_id'] == play_id)
    ]
    
    supp_info = supp.iloc[0] if len(supp) > 0 else None
    
    input_frames = sorted(input_data[
        (input_data['game_id'] == game_id) & 
        (input_data['play_id'] == play_id)
    ]['frame_id'].unique().tolist())
    
    output_frames = sorted(output_data[
        (output_data['game_id'] == game_id) & 
        (output_data['play_id'] == play_id)
    ]['frame_id'].unique().tolist())
    
    if len(input_frames) == 0:
        return None
    
    best = preds[preds['is_best_option'] == True]
    if len(best) == 0:
        best = preds.loc[preds['catch_probability'].idxmax()]
    else:
        best = best.iloc[0]
    
    best_option = {
        'nfl_id': int(best['nfl_id']),
        'receiver_name': best.get('receiver_name', 'Unknown'),
        'receiver_position': best.get('receiver_position', '??'),
        'predicted_route': best.get('predicted_route', '?'),
        'catch_probability': float(best['catch_probability'])
    }
    
    actual_target = None
    targeted = preds[preds['is_targeted'] == True]
    if len(targeted) > 0:
        t = targeted.iloc[0]
        actual_target = {
            'nfl_id': int(t['nfl_id']),
            'receiver_name': t.get('receiver_name', 'Unknown'),
            'receiver_position': t.get('receiver_position', '??'),
            'predicted_route': t.get('predicted_route', '?'),
            'catch_probability': float(t['catch_probability'])
        }
    
    timing_frames = int(play_info['timing_frames'])
    pause_idx = min(timing_frames, len(input_frames) - 1)
    pause_frame = input_frames[pause_idx]
    
    # Get QB position at first frame to determine play direction
    first_frame = input_frames[0]
    qb_data = input_data[
        (input_data['game_id'] == game_id) & 
        (input_data['play_id'] == play_id) &
        (input_data['frame_id'] == first_frame) &
        (input_data['player_role'] == 'Passer')
    ]
    
    if len(qb_data) > 0:
        qb_pos = [float(qb_data['x'].iloc[0]), float(qb_data['y'].iloc[0])]
    else:
        qb_pos = [25.0, 26.65]
    
    # Get average offensive and defensive positions to determine play direction
    off_data = input_data[
        (input_data['game_id'] == game_id) & 
        (input_data['play_id'] == play_id) &
        (input_data['frame_id'] == first_frame) &
        (input_data['player_side'] == 'Offense')
    ]
    
    def_data = input_data[
        (input_data['game_id'] == game_id) & 
        (input_data['play_id'] == play_id) &
        (input_data['frame_id'] == first_frame) &
        (input_data['player_side'] == 'Defense')
    ]
    
    if len(off_data) > 0 and len(def_data) > 0:
        avg_off_x = off_data['x'].mean()
        avg_def_x = def_data['x'].mean()
        # Offense attacks TOWARD the defense
        # If defense avg X is greater, offense attacks right (toward higher X)
        # If defense avg X is lower, offense attacks left (toward lower X)
        attacking_right = avg_def_x > avg_off_x
    else:
        attacking_right = True  # Default assumption
    
    preds_sorted = preds.sort_values('total_catches_season', ascending=False)
    receiver_rankings = {
        int(row['nfl_id']): idx + 1 
        for idx, (_, row) in enumerate(preds_sorted.iterrows())
    }
    
    result = play_info.get('play_result', 'UNKNOWN')
    qb_name = play_info.get('qb_name', 'QB')
    
    # Extract field marking and score info from supplementary data
    if supp_info is not None:
        possession_team = supp_info.get('possession_team', 'OFF')
        defensive_team = supp_info.get('defensive_team', 'DEF')
        
        yards_to_go_raw = supp_info.get('yards_to_go', 10)
        yards_to_go = int(yards_to_go_raw) if not pd.isna(yards_to_go_raw) else 10
        
        yards_gained_raw = supp_info.get('yards_gained', 0)
        yards_gained = int(yards_gained_raw) if not pd.isna(yards_gained_raw) else 0
        
        # Get EP and EPA
        ep_raw = supp_info.get('expected_points', 0)
        ep = float(ep_raw) if not pd.isna(ep_raw) else 0.0
        
        epa_raw = supp_info.get('expected_points_added', 0)
        epa = float(epa_raw) if not pd.isna(epa_raw) else 0.0
        
        # Determine scores based on home/away
        home_team = supp_info.get('home_team_abbr', 'HOME')
        visitor_team = supp_info.get('visitor_team_abbr', 'AWAY')
        
        home_score_raw = supp_info.get('pre_snap_home_score', 0)
        home_score = int(home_score_raw) if not pd.isna(home_score_raw) else 0
        
        visitor_score_raw = supp_info.get('pre_snap_visitor_score', 0)
        visitor_score = int(visitor_score_raw) if not pd.isna(visitor_score_raw) else 0
        
        if possession_team == home_team:
            off_score = home_score
            def_score = visitor_score
        else:
            off_score = visitor_score
            def_score = home_score
        
        # Get yardline info
        yardline_side = supp_info.get('yardline_side', '')
        yardline_number_raw = supp_info.get('yardline_number', 50)
        yardline_number = int(yardline_number_raw) if not pd.isna(yardline_number_raw) else 50
        
        # Determine if on own territory or opponent territory
        on_own_territory = (yardline_side == possession_team) or pd.isna(yardline_side) or yardline_side == ''
        
        # Calculate LOS based on attacking direction and territory
        # Field: x=10 is left goal line, x=110 is right goal line
        # yardline_number = yards from the relevant team's goal line
        if attacking_right:
            # Attacking toward x=110 (right endzone)
            # Own endzone is at x=10, opponent's at x=110
            if on_own_territory:
                # Own 30 = 30 yards from own goal (x=10), so x = 10 + 30
                los_x = 10 + yardline_number
            else:
                # Opponent's 30 = 30 yards from opponent's goal (x=110), so x = 110 - 30
                los_x = 110 - yardline_number
        else:
            # Attacking toward x=10 (left endzone)
            # Own endzone is at x=110, opponent's at x=10
            if on_own_territory:
                # Own 30 = 30 yards from own goal (x=110), so x = 110 - 30
                los_x = 110 - yardline_number
            else:
                # Opponent's 30 = 30 yards from opponent's goal (x=10), so x = 10 + 30
                los_x = 10 + yardline_number
        
        # First down line based on play direction
        if attacking_right:
            first_down_x = los_x + yards_to_go
            if first_down_x > 110:
                first_down_x = 110  # Cap at goal line
            is_touchdown = (los_x + yards_gained) >= 110
        else:
            first_down_x = los_x - yards_to_go
            if first_down_x < 10:
                first_down_x = 10  # Cap at goal line
            is_touchdown = (los_x - yards_gained) <= 10
    else:
        possession_team = 'OFF'
        defensive_team = 'DEF'
        off_score = 0
        def_score = 0
        # Default to middle of field
        los_x = 60
        if attacking_right:
            first_down_x = los_x + 10
        else:
            first_down_x = los_x - 10
        yards_to_go = 10
        yards_gained = 0
        is_touchdown = False
        ep = 0.0
        epa = 0.0
    
    return {
        'game_id': game_id,
        'play_id': play_id,
        'predictions': preds,
        'play_info': play_info,
        'concept': play_info.get('concept_final', 'UNKNOWN'),
        'timing_frames': timing_frames,
        'pause_frame': pause_frame,
        'input_frames': input_frames,
        'output_frames': output_frames,
        'best_option': best_option,
        'actual_target': actual_target,
        'result': result,
        'qb_pos': qb_pos,
        'qb_name': qb_name,
        'receiver_rankings': receiver_rankings,
        'receiver_ids': preds['nfl_id'].tolist(),
        # Field markings
        'los_x': los_x,
        'first_down_x': first_down_x,
        'yards_to_go': yards_to_go,
        'yards_gained': yards_gained,
        'is_touchdown': is_touchdown,
        'attacking_right': attacking_right,
        # Teams and scores
        'possession_team': possession_team,
        'defensive_team': defensive_team,
        'off_score': off_score,
        'def_score': def_score,
        # Expected points
        'ep': ep,
        'epa': epa
    }

# =============================================================================
# CELL 4: DASH APP
# =============================================================================

def precompute_play_frames(game_id, play_id, input_frames, output_frames):
    """Precompute all player positions for every frame of a play."""
    all_frames = sorted(set(input_frames + output_frames))
    frame_data = {}
    
    for frame_id in all_frames:
        frame_tracking = input_data[
            (input_data['game_id'] == game_id) & 
            (input_data['play_id'] == play_id) &
            (input_data['frame_id'] == frame_id)
        ]
        
        if len(frame_tracking) == 0:
            frame_tracking = output_data[
                (output_data['game_id'] == game_id) & 
                (output_data['play_id'] == play_id) &
                (output_data['frame_id'] == frame_id)
            ]
        
        if len(frame_tracking) > 0:
            players = []
            for _, row in frame_tracking.iterrows():
                players.append({
                    'nfl_id': int(row['nfl_id']),
                    'x': float(row['x']),
                    'y': float(row['y']),
                    'player_role': row.get('player_role', ''),
                    'player_side': row.get('player_side', '')
                })
            frame_data[frame_id] = players
    
    return frame_data


def create_app():
    """Create and return a fresh Dash app instance."""
    
    POSITION_COLORS = {
        'WR': '#2196f3',
        'TE': '#ffd700',
        'RB': '#4caf50',
        'FB': '#4caf50',
    }
    DEFAULT_COLOR = '#9e9e9e'
    
    SPEED_OPTIONS = [
        {'label': '0.5x', 'value': 0.5},
        {'label': '0.75x', 'value': 0.75},
        {'label': '1.0x', 'value': 1.0},
        {'label': '1.25x', 'value': 1.25},
        {'label': '1.5x', 'value': 1.5},
    ]
    TIMER_OPTIONS = [
        {'label': '2s', 'value': 2},
        {'label': '5s', 'value': 5},
        {'label': '10s', 'value': 10},
        {'label': '∞', 'value': 0},
    ]
    BASE_FRAME_DURATION = 120
    
    def create_field_figure(los_x=None, first_down_x=None):
        """Create empty football field figure with optional field markings."""
        fig = go.Figure()
        
        field_width = 53.3
        field_length = 120
        
        fig.add_shape(type="rect", x0=0, y0=0, x1=field_length, y1=field_width,
                      fillcolor="#2e7d32", line=dict(width=0), layer="below")
        
        fig.add_shape(type="rect", x0=0, y0=0, x1=10, y1=field_width,
                      fillcolor="#1b5e20", line=dict(color="white", width=2), layer="below")
        fig.add_shape(type="rect", x0=110, y0=0, x1=120, y1=field_width,
                      fillcolor="#1b5e20", line=dict(color="white", width=2), layer="below")
        
        for yard in range(10, 111, 5):
            width = 2 if yard % 10 == 0 else 1
            fig.add_shape(type="line", x0=yard, y0=0, x1=yard, y1=field_width,
                          line=dict(color="white", width=width), layer="below")
        
        for yard in range(10, 100, 10):
            display_num = yard if yard <= 50 else 100 - yard
            fig.add_annotation(x=yard+10, y=field_width/2, text=str(display_num),
                              font=dict(size=20, color="white"), showarrow=False, opacity=0.5)
        
        # Line of scrimmage (blue)
        if los_x is not None:
            fig.add_shape(type="line", x0=los_x, y0=0, x1=los_x, y1=field_width,
                          line=dict(color="#2196f3", width=3), layer="below")
        
        # First down line (yellow)
        if first_down_x is not None and first_down_x <= 110:
            fig.add_shape(type="line", x0=first_down_x, y0=0, x1=first_down_x, y1=field_width,
                          line=dict(color="#ffd700", width=3), layer="below")
        
        fig.update_layout(
            xaxis=dict(range=[-5, 125], showgrid=False, zeroline=False, showticklabels=False, fixedrange=True),
            yaxis=dict(range=[-5, field_width+5], showgrid=False, zeroline=False, showticklabels=False, scaleanchor="x", fixedrange=True),
            plot_bgcolor='rgba(0,0,0,0)',
            paper_bgcolor='#1a1a1a',
            margin=dict(l=10, r=10, t=10, b=10),
            height=500,
            width=1100,
            autosize=False,
            showlegend=True,
            uirevision='field',
            legend=dict(
                bgcolor='rgba(30,30,30,0.9)',
                bordercolor='white', borderwidth=1,
                font=dict(color='white', size=11),
                x=1.01, y=1.0, xanchor='left', yanchor='top'
            )
        )
        
        return fig
    
    def build_animated_figure(frame_data, frame_list, predictions, receiver_rankings, 
                              start_frame, end_frame, speed_multiplier=1.0,
                              phase='running', qb_pos=None, actual_nfl_id=None, result=None,
                              qb_name='QB', los_x=None, first_down_x=None):
        """Build a figure with Plotly animation frames."""
        fig = create_field_figure(los_x=los_x, first_down_x=first_down_x)
        
        receiver_info = {row['nfl_id']: row for _, row in predictions.iterrows()}
        receiver_ids = set(receiver_info.keys())
        
        frames_to_animate = [f for f in frame_list if start_frame <= f <= end_frame]
        if not frames_to_animate:
            return fig, BASE_FRAME_DURATION
        
        first_frame_id = frames_to_animate[0]
        first_frame_data = frame_data.get(first_frame_id, [])
        
        def organize_players(players):
            defense_x, defense_y = [], []
            qb_x, qb_y = [], []
            other_off_x, other_off_y = [], []
            receivers = {rid: {'x': None, 'y': None} for rid in receiver_ids}
            
            for p in players:
                nfl_id = p['nfl_id']
                if p['player_side'] == 'Defense':
                    defense_x.append(p['x'])
                    defense_y.append(p['y'])
                elif p['player_role'] == 'Passer':
                    qb_x.append(p['x'])
                    qb_y.append(p['y'])
                elif nfl_id in receiver_ids:
                    receivers[nfl_id] = {'x': p['x'], 'y': p['y']}
                elif p['player_side'] == 'Offense':
                    other_off_x.append(p['x'])
                    other_off_y.append(p['y'])
            
            return defense_x, defense_y, qb_x, qb_y, receivers, other_off_x, other_off_y
        
        d_x, d_y, q_x, q_y, recs, o_x, o_y = organize_players(first_frame_data)
        
        # Trace 0: Defense
        fig.add_trace(go.Scatter(
            x=d_x, y=d_y, mode='markers',
            marker=dict(size=10, color='#ef5350', line=dict(width=1, color='white')),
            name='Defense', hoverinfo='skip', showlegend=True
        ))
        
        # Trace 1: QB
        fig.add_trace(go.Scatter(
            x=q_x, y=q_y, mode='markers',
            marker=dict(size=14, color='white', symbol='square', line=dict(width=2, color='#333')),
            name=qb_name, hoverinfo='skip', showlegend=True
        ))
        
        # Traces 2+: Receivers
        receiver_id_list = sorted(receiver_ids)
        for rid in receiver_id_list:
            pred = receiver_info[rid]
            position = pred.get('receiver_position', '??')
            receiver_name = pred.get('receiver_name', 'Unknown')
            route = pred.get('predicted_route', '?')
            rank_num = receiver_rankings.get(rid, 99)
            
            name_parts = str(receiver_name).split() if receiver_name else ['?']
            if len(name_parts) >= 2:
                display_name = f"{name_parts[0][0]}. {name_parts[-1]}"
            else:
                display_name = str(receiver_name)
            
            base_color = POSITION_COLORS.get(position, DEFAULT_COLOR)
            legend_name = f"{rank_num}. {display_name} ({position})"
            
            rx = [recs[rid]['x']] if recs[rid]['x'] is not None else []
            ry = [recs[rid]['y']] if recs[rid]['y'] is not None else []
            
            fig.add_trace(go.Scatter(
                x=rx, y=ry, mode='markers+text',
                marker=dict(size=16, color=base_color, line=dict(width=1, color='white')),
                text=str(rank_num) if rank_num <= 5 else '',
                textposition='middle center',
                textfont=dict(size=9, color='white', family='Arial Black'),
                name=legend_name,
                customdata=[rid],
                hovertemplate=f"{display_name}<br>{route}<extra></extra>",
                showlegend=True
            ))
        
        # Other offense
        fig.add_trace(go.Scatter(
            x=o_x, y=o_y, mode='markers',
            marker=dict(size=8, color='#607d8b', line=dict(width=1, color='white')),
            hoverinfo='skip', showlegend=False
        ))
        
        # Ball (diamond marker)
        fig.add_trace(go.Scatter(
            x=[], y=[], mode='markers',
            marker=dict(size=18, color='#8B4513', symbol='diamond-wide', 
                       line=dict(width=2, color='white')),
            name='Ball', hoverinfo='skip', showlegend=False
        ))
        
        # Build frames
        frames = []
        for i, frame_id in enumerate(frames_to_animate):
            players = frame_data.get(frame_id, [])
            d_x, d_y, q_x, q_y, recs, o_x, o_y = organize_players(players)
            
            ball_x, ball_y = [], []
            if phase == 'post-throw' and qb_pos and actual_nfl_id:
                progress = i / max(len(frames_to_animate) - 1, 1)
                actual_rec = recs.get(actual_nfl_id, {'x': None, 'y': None})
                if actual_rec['x'] is not None:
                    ball_x = [qb_pos[0] + (actual_rec['x'] - qb_pos[0]) * progress]
                    ball_y = [qb_pos[1] + (actual_rec['y'] - qb_pos[1]) * progress]
            
            frame_traces = [
                go.Scatter(x=d_x, y=d_y),
                go.Scatter(x=q_x, y=q_y),
            ]
            
            for rid in receiver_id_list:
                rx = [recs[rid]['x']] if recs[rid]['x'] is not None else []
                ry = [recs[rid]['y']] if recs[rid]['y'] is not None else []
                frame_traces.append(go.Scatter(x=rx, y=ry))
            
            frame_traces.append(go.Scatter(x=o_x, y=o_y))
            frame_traces.append(go.Scatter(x=ball_x, y=ball_y))
            
            frames.append(go.Frame(data=frame_traces, name=str(frame_id)))
        
        fig.frames = frames
        
        frame_duration = int(BASE_FRAME_DURATION / speed_multiplier)
        transition_duration = int(frame_duration / 2)
        
        fig.update_layout(
            updatemenus=[{
                'type': 'buttons',
                'showactive': False,
                'visible': False,
                'buttons': [{
                    'label': 'Play',
                    'method': 'animate',
                    'args': [None, {
                        'frame': {'duration': frame_duration, 'redraw': True},
                        'fromcurrent': True,
                        'transition': {'duration': transition_duration, 'easing': 'linear'}
                    }]
                }]
            }]
        )
        
        return fig, frame_duration
    
    def build_static_figure(frame_data, frame_id, predictions, receiver_rankings,
                           show_probs=False, selected_id=None, selection_correct=None,
                           show_ball=False, ball_pos=None, ball_result=None,
                           actual_receiver_id=None, qb_name='QB', los_x=None, first_down_x=None,
                           show_routes=False, attacking_right=True):
        """Build a static figure for paused/selection states."""
        fig = create_field_figure(los_x=los_x, first_down_x=first_down_x)
        
        receiver_info = {row['nfl_id']: row for _, row in predictions.iterrows()}
        receiver_ids = set(receiver_info.keys())
        
        players = frame_data.get(frame_id, [])
        
        defense = [p for p in players if p['player_side'] == 'Defense']
        offense = [p for p in players if p['player_side'] == 'Offense']
        
        if defense:
            fig.add_trace(go.Scatter(
                x=[p['x'] for p in defense], y=[p['y'] for p in defense],
                mode='markers',
                marker=dict(size=10, color='#ef5350', line=dict(width=1, color='white')),
                name='Defense', hoverinfo='skip', showlegend=True
            ))
        
        # Route labels (only shown pre-snap) - positioned AWAY from defense
        # If attacking right, defense is to the right, so labels go LEFT
        # If attacking left, defense is to the left, so labels go RIGHT
        if show_routes:
            label_x_offset = -2.5 if attacking_right else 2.5
            x_anchor = 'right' if attacking_right else 'left'
            for p in offense:
                nfl_id = p['nfl_id']
                if nfl_id in receiver_ids:
                    pred = receiver_info[nfl_id]
                    route = pred.get('predicted_route', '?')
                    fig.add_annotation(
                        x=p['x'] + label_x_offset, y=p['y'],
                        text=route,
                        font=dict(size=9, color='white', family='Arial'),
                        showarrow=False,
                        bgcolor='rgba(0,0,0,0.6)',
                        borderpad=2,
                        xanchor=x_anchor
                    )
        
        for p in offense:
            nfl_id = p['nfl_id']
            player_role = p.get('player_role', '')
            
            if player_role == 'Passer':
                fig.add_trace(go.Scatter(
                    x=[p['x']], y=[p['y']], mode='markers',
                    marker=dict(size=14, color='white', symbol='square',
                               line=dict(width=2, color='#333')),
                    name=qb_name, hoverinfo='skip', showlegend=True
                ))
            elif nfl_id in receiver_ids:
                pred = receiver_info[nfl_id]
                prob = pred['catch_probability']
                is_best = pred.get('is_best_option', False)
                route = pred.get('predicted_route', '?')
                position = pred.get('receiver_position', '??')
                receiver_name = pred.get('receiver_name', 'Unknown')
                rank_num = receiver_rankings.get(nfl_id, 99)
                
                name_parts = str(receiver_name).split() if receiver_name else ['?']
                if len(name_parts) >= 2:
                    display_name = f"{name_parts[0][0]}. {name_parts[-1]}"
                else:
                    display_name = str(receiver_name)
                
                base_color = POSITION_COLORS.get(position, DEFAULT_COLOR)
                
                # Default styling
                color = base_color
                line_color = 'white'
                line_width = 1
                
                # After user makes selection
                if selected_id is not None:
                    if nfl_id == selected_id:
                        color = base_color
                        line_color = '#4caf50' if selection_correct else '#f44336'
                        line_width = 5
                    elif is_best and show_probs:
                        color = base_color
                        line_color = '#ffd700'
                        line_width = 4
                # Post-throw: highlight actual target
                elif show_ball and actual_receiver_id is not None and nfl_id == actual_receiver_id:
                    color = base_color
                    line_color = '#4caf50' if ball_result == 'CAUGHT' else '#f44336'
                    line_width = 4
                
                hover = f"{display_name}<br>{route}"
                legend_name = f"{rank_num}. {display_name} ({position})"
                
                fig.add_trace(go.Scatter(
                    x=[p['x']], y=[p['y']], mode='markers+text',
                    marker=dict(size=16, color=color, line=dict(width=line_width, color=line_color)),
                    text=str(rank_num) if rank_num <= 5 else '',
                    textposition='middle center',
                    textfont=dict(size=9, color='white', family='Arial Black'),
                    name=legend_name,
                    customdata=[nfl_id],
                    hovertemplate=hover + "<extra></extra>",
                    showlegend=True
                ))
            else:
                fig.add_trace(go.Scatter(
                    x=[p['x']], y=[p['y']], mode='markers',
                    marker=dict(size=8, color='#607d8b', line=dict(width=1, color='white')),
                    hoverinfo='skip', showlegend=False
                ))
        
        if show_ball and ball_pos is not None:
            fig.add_trace(go.Scatter(
                x=[ball_pos[0]], y=[ball_pos[1]], mode='markers',
                marker=dict(size=18, color='#8B4513', symbol='diamond-wide',
                           line=dict(width=2, color='white')),
                hoverinfo='skip', showlegend=False
            ))
        
        return fig
    
    # Create Dash app
    app = Dash(__name__)
    
    app.layout = html.Div([
        html.H1("QB Decision Trainer", style={'textAlign': 'center', 'color': 'white', 'margin': '0px 0px 0px 0px', 'padding': '0px'}),
        
        html.Div(id='play-info', style={'textAlign': 'center', 'color': 'white', 'fontSize': '18px', 'marginBottom': '2px'}),
        
        # Scoreboard above field
        html.Div(id='scoreboard', children=[
            # Left team (defense - away from endzone offense is attacking)
            html.Div([
                html.Span("DEF", style={
                    'backgroundColor': 'white', 'color': 'black', 'padding': '5px 15px',
                    'fontWeight': 'bold', 'fontSize': '16px', 'borderRadius': '3px 0 0 3px'
                }),
                html.Span("0", style={
                    'backgroundColor': '#ef5350', 'color': 'white', 'padding': '5px 15px',
                    'fontWeight': 'bold', 'fontSize': '16px', 'borderRadius': '0 3px 3px 0'
                })
            ], style={'display': 'inline-block', 'marginRight': '20px'}),
            # Right team (offense - attacking right endzone)
            html.Div([
                html.Span("0", id='off-score-display', style={
                    'backgroundColor': '#4caf50', 'color': 'white', 'padding': '5px 15px',
                    'fontWeight': 'bold', 'fontSize': '16px', 'borderRadius': '3px 0 0 3px'
                }),
                html.Span("OFF", style={
                    'backgroundColor': 'white', 'color': 'black', 'padding': '5px 15px',
                    'fontWeight': 'bold', 'fontSize': '16px', 'borderRadius': '0 3px 3px 0'
                })
            ], style={'display': 'inline-block'})
        ], style={'textAlign': 'center', 'marginBottom': '2px'}),
        
        # EP/EPA legend (top left, above field corner) - hidden until selection
        html.Div(id='ep-display', children=[
            html.Span("EP: ", style={'color': '#888', 'fontSize': '11px'}),
            html.Span("0.00", style={'color': 'white', 'fontSize': '11px', 'fontWeight': 'bold', 'marginRight': '15px'}),
            html.Span("EPA: ", style={'color': '#888', 'fontSize': '11px'}),
            html.Span("0.00", style={'color': 'white', 'fontSize': '11px', 'fontWeight': 'bold'}),
        ], style={'position': 'absolute', 'left': '160px', 'top': '100px', 'backgroundColor': 'rgba(30,30,30,0.9)',
                  'padding': '4px 10px', 'borderRadius': '4px', 'border': '1px solid white', 'zIndex': '100', 'display': 'none'}),
        
        # Main content area: field + feedback side by side
        html.Div([
            dcc.Graph(id='field', config={'displayModeBar': False}, 
                     style={'height': '500px', 'width': '1100px'}),
            
            # Symbol legend (inside field container for proper positioning)
            html.Div([
                html.Span("●", style={'color': '#ef5350', 'fontSize': '12px', 'marginRight': '1px'}),
                html.Span("DEF", style={'color': 'white', 'fontSize': '10px', 'marginRight': '8px'}),
                html.Span("●", style={'color': '#4fc3f7', 'fontSize': '12px', 'marginRight': '1px'}),
                html.Span("WR", style={'color': 'white', 'fontSize': '10px', 'marginRight': '8px'}),
                html.Span("●", style={'color': '#ffeb3b', 'fontSize': '12px', 'marginRight': '1px'}),
                html.Span("TE", style={'color': 'white', 'fontSize': '10px', 'marginRight': '8px'}),
                html.Span("●", style={'color': '#4caf50', 'fontSize': '12px', 'marginRight': '1px'}),
                html.Span("RB", style={'color': 'white', 'fontSize': '10px', 'marginRight': '8px'}),
                html.Span("■", style={'color': 'white', 'fontSize': '10px', 'marginRight': '1px'}),
                html.Span("QB", style={'color': 'white', 'fontSize': '10px', 'marginRight': '8px'}),
                html.Span("│", style={'color': '#2196f3', 'fontSize': '12px', 'fontWeight': 'bold', 'marginRight': '1px'}),
                html.Span("LOS", style={'color': 'white', 'fontSize': '10px', 'marginRight': '8px'}),
                html.Span("│", style={'color': '#ffd700', 'fontSize': '12px', 'fontWeight': 'bold', 'marginRight': '1px'}),
                html.Span("1st", style={'color': 'white', 'fontSize': '10px'}),
            ], style={'position': 'absolute', 'right': '320px', 'top': '10px', 'backgroundColor': 'rgba(30,30,30,0.9)',
                      'padding': '4px 8px', 'borderRadius': '4px', 'border': '1px solid white', 'zIndex': '100'}),
            
            html.Div(id='feedback-panel', style={
                'display': 'none',
                'width': '170px',
                'position': 'absolute',
                'right': '90px',
                'top': '168px',
                'zIndex': '1000'
            }),
        ], style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'flex-start', 'position': 'relative'}),
        
        # Controls below
        html.Div([
            html.Button('▶ Play', id='play-btn', n_clicks=0, 
                       style={'fontSize': '20px', 'padding': '10px 30px', 'marginRight': '10px'}),
            html.Button('Next Play', id='next-btn', n_clicks=0,
                       style={'fontSize': '16px', 'padding': '10px 20px', 'marginRight': '20px'}),
            
            html.Label('Speed: ', style={'color': 'white', 'fontSize': '16px', 'marginRight': '5px'}),
            dcc.Dropdown(
                id='speed-dropdown',
                options=SPEED_OPTIONS,
                value=1.0,
                clearable=False,
                style={'width': '80px', 'display': 'inline-block', 'verticalAlign': 'middle'}
            ),
            
            html.Label('Timer: ', style={'color': 'white', 'fontSize': '16px', 'marginLeft': '20px', 'marginRight': '5px'}),
            dcc.Dropdown(
                id='timer-dropdown',
                options=TIMER_OPTIONS,
                value=5,
                clearable=False,
                style={'width': '70px', 'display': 'inline-block', 'verticalAlign': 'middle'}
            ),
            
            html.Span(id='timer-display', style={'color': '#ffd700', 'marginLeft': '10px', 'fontSize': '18px', 'fontWeight': 'bold', 'minWidth': '30px'}),
            
            html.Span(id='frame-display', style={'color': 'white', 'marginLeft': '20px', 'fontSize': '16px'})
        ], style={'textAlign': 'center', 'marginTop': '0px', 'display': 'flex', 
                  'justifyContent': 'center', 'alignItems': 'center'}),
        
        dcc.Store(id='play-data'),
        dcc.Store(id='frame-data-store'),
        dcc.Store(id='current-frame', data=1),
        dcc.Store(id='app-phase', data='initial'),
        dcc.Store(id='user-selection'),
        dcc.Store(id='selection-correct'),
        dcc.Store(id='animation-trigger', data=0),
        dcc.Store(id='frame-duration', data=BASE_FRAME_DURATION),
        dcc.Store(id='timer-remaining', data=None),
        dcc.Store(id='timer-active', data=False),
        
        dcc.Interval(id='animation-check', interval=100, disabled=True),
        dcc.Interval(id='timer-interval', interval=1000, disabled=True)
        
    ], style={'backgroundColor': '#1a1a1a', 'minHeight': '100vh', 'padding': '10px', 'position': 'relative'})
    
    # Clientside callback to trigger animation
    app.clientside_callback(
        """
        function(trigger, phase, frameDuration) {
            if (trigger && (phase === 'running' || phase === 'post-throw')) {
                setTimeout(function() {
                    var plotDiv = document.querySelector("#field .js-plotly-plot");
                    if (plotDiv && plotDiv._fullLayout && plotDiv._fullData) {
                        try {
                            Plotly.animate(plotDiv, null, {
                                frame: {duration: frameDuration, redraw: true},
                                transition: {duration: frameDuration/2, easing: 'linear'},
                                fromcurrent: true,
                                mode: 'immediate'
                            });
                        } catch(e) {
                            console.log('Animation error:', e);
                        }
                    } else {
                        console.log('Plotly element not ready');
                    }
                }, 350);
            }
            return window.dash_clientside.no_update;
        }
        """,
        Output('field', 'id'),
        Input('animation-trigger', 'data'),
        State('app-phase', 'data'),
        State('frame-duration', 'data'),
        prevent_initial_call=True
    )
    
    @app.callback(
        Output('play-data', 'data'),
        Output('frame-data-store', 'data'),
        Output('play-info', 'children'),
        Output('current-frame', 'data'),
        Output('app-phase', 'data'),
        Output('user-selection', 'data'),
        Output('selection-correct', 'data'),
        Output('feedback-panel', 'style'),
        Output('scoreboard', 'children'),
        Output('ep-display', 'children'),
        Output('ep-display', 'style'),
        Output('timer-remaining', 'data'),
        Output('timer-display', 'children'),
        Input('next-btn', 'n_clicks'),
        prevent_initial_call='initial_duplicate'
    )
    def load_new_play(n_clicks):
        sample = plays_app.sample(1).iloc[0]
        game_id = int(sample['game_id'])
        play_id = int(sample['play_id'])
        
        play = load_play_for_coaching(game_id, play_id)
        
        default_scoreboard = [
            html.Div([
                html.Span("DEF", style={
                    'backgroundColor': 'white', 'color': 'black', 'padding': '5px 15px',
                    'fontWeight': 'bold', 'fontSize': '16px', 'borderRadius': '3px 0 0 3px'
                }),
                html.Span("0", style={
                    'backgroundColor': '#ef5350', 'color': 'white', 'padding': '5px 15px',
                    'fontWeight': 'bold', 'fontSize': '16px', 'borderRadius': '0 3px 3px 0'
                })
            ], style={'display': 'inline-block', 'marginRight': '20px'}),
            html.Div([
                html.Span("0", style={
                    'backgroundColor': '#4caf50', 'color': 'white', 'padding': '5px 15px',
                    'fontWeight': 'bold', 'fontSize': '16px', 'borderRadius': '3px 0 0 3px'
                }),
                html.Span("OFF", style={
                    'backgroundColor': 'white', 'color': 'black', 'padding': '5px 15px',
                    'fontWeight': 'bold', 'fontSize': '16px', 'borderRadius': '0 3px 3px 0'
                })
            ], style={'display': 'inline-block'})
        ]
        
        default_ep_display = [
            html.Span("EP: ", style={'color': '#888', 'fontSize': '11px'}),
            html.Span("0.00", style={'color': 'white', 'fontSize': '11px', 'fontWeight': 'bold', 'marginRight': '15px'}),
            html.Span("EPA: ", style={'color': '#888', 'fontSize': '11px'}),
            html.Span("0.00", style={'color': 'white', 'fontSize': '11px', 'fontWeight': 'bold'}),
        ]
        
        ep_hidden_style = {'position': 'absolute', 'left': '160px', 'top': '100px', 'backgroundColor': 'rgba(30,30,30,0.9)',
                           'padding': '4px 10px', 'borderRadius': '4px', 'border': '1px solid white', 'zIndex': '100', 'display': 'none'}
        
        if play is None:
            return None, None, "Error loading play", 1, 'initial', None, None, {'display': 'none'}, default_scoreboard, default_ep_display, ep_hidden_style, None, ""
        
        frame_data = precompute_play_frames(game_id, play_id, play['input_frames'], play['output_frames'])
        
        play_json = {
            'game_id': game_id,
            'play_id': play_id,
            'concept': play['concept'],
            'timing_frames': play['timing_frames'],
            'pause_frame': play['pause_frame'],
            'input_frames': play['input_frames'],
            'output_frames': play['output_frames'],
            'receiver_ids': [int(x) for x in play['receiver_ids']],
            'best_nfl_id': play['best_option']['nfl_id'],
            'best_name': play['best_option']['receiver_name'],
            'best_route': play['best_option']['predicted_route'],
            'best_prob': play['best_option']['catch_probability'],
            'result': play['result'],
            'qb_pos': play['qb_pos'],
            'qb_name': play['qb_name'],
            'receiver_rankings': {str(k): int(v) for k, v in play['receiver_rankings'].items()},
            # Field markings
            'los_x': play['los_x'],
            'first_down_x': play['first_down_x'],
            'yards_to_go': play['yards_to_go'],
            'yards_gained': play['yards_gained'],
            'is_touchdown': play['is_touchdown'],
            'attacking_right': play['attacking_right'],
            # Teams and scores
            'possession_team': play['possession_team'],
            'defensive_team': play['defensive_team'],
            'off_score': play['off_score'],
            'def_score': play['def_score'],
            # Expected points
            'ep': play['ep'],
            'epa': play['epa']
        }
        
        if play['actual_target'] is not None:
            play_json['actual_nfl_id'] = play['actual_target']['nfl_id']
            play_json['actual_name'] = play['actual_target']['receiver_name']
            play_json['actual_route'] = play['actual_target']['predicted_route']
            play_json['actual_prob'] = play['actual_target']['catch_probability']
        
        frame_data_json = {str(k): v for k, v in frame_data.items()}
        
        info = f"{play['qb_name']} | {play['concept']} | {play['yards_to_go']} to go"
        start_frame = play['input_frames'][0] if play['input_frames'] else 1
        
        # Build scoreboard with actual team names and scores
        scoreboard = [
            html.Div([
                html.Span(play['defensive_team'], style={
                    'backgroundColor': 'white', 'color': 'black', 'padding': '5px 15px',
                    'fontWeight': 'bold', 'fontSize': '16px', 'borderRadius': '3px 0 0 3px'
                }),
                html.Span(str(play['def_score']), style={
                    'backgroundColor': '#ef5350', 'color': 'white', 'padding': '5px 15px',
                    'fontWeight': 'bold', 'fontSize': '16px', 'borderRadius': '0 3px 3px 0'
                })
            ], style={'display': 'inline-block', 'marginRight': '20px'}),
            html.Div([
                html.Span(str(play['off_score']), style={
                    'backgroundColor': '#4caf50', 'color': 'white', 'padding': '5px 15px',
                    'fontWeight': 'bold', 'fontSize': '16px', 'borderRadius': '3px 0 0 3px'
                }),
                html.Span(play['possession_team'], style={
                    'backgroundColor': 'white', 'color': 'black', 'padding': '5px 15px',
                    'fontWeight': 'bold', 'fontSize': '16px', 'borderRadius': '0 3px 3px 0'
                })
            ], style={'display': 'inline-block'})
        ]
        
        # EP/EPA display
        ep_display = [
            html.Span("EP: ", style={'color': '#888', 'fontSize': '11px'}),
            html.Span(f"{play['ep']:.2f}", style={'color': 'white', 'fontSize': '11px', 'fontWeight': 'bold', 'marginRight': '15px'}),
            html.Span("EPA: ", style={'color': '#888', 'fontSize': '11px'}),
            html.Span(f"{play['epa']:.2f}", style={'color': '#4caf50' if play['epa'] >= 0 else '#ef5350', 'fontSize': '11px', 'fontWeight': 'bold'}),
        ]
        
        return play_json, frame_data_json, info, start_frame, 'initial', None, None, {'display': 'none'}, scoreboard, ep_display, ep_hidden_style, None, ""
    
    @app.callback(
        Output('frame-duration', 'data'),
        Input('speed-dropdown', 'value'),
    )
    def update_speed(speed_multiplier):
        return int(BASE_FRAME_DURATION / speed_multiplier)
    
    @app.callback(
        Output('field', 'figure'),
        Output('frame-display', 'children'),
        Input('app-phase', 'data'),
        Input('play-data', 'data'),
        Input('frame-data-store', 'data'),
        Input('current-frame', 'data'),
        Input('user-selection', 'data'),
        Input('selection-correct', 'data'),
        Input('speed-dropdown', 'value'),
    )
    def update_field(phase, play_data, frame_data_json, current_frame, selected_id, selection_correct, speed):
        if play_data is None or frame_data_json is None:
            fig = create_field_figure()
            return fig, ""
        
        frame_data = {int(k): v for k, v in frame_data_json.items()}
        receiver_rankings = {int(k): v for k, v in play_data.get('receiver_rankings', {}).items()}
        qb_name = play_data.get('qb_name', 'QB')
        los_x = play_data.get('los_x')
        first_down_x = play_data.get('first_down_x')
        attacking_right = play_data.get('attacking_right', True)
        
        preds = receivers_app[
            (receivers_app['game_id'] == play_data['game_id']) & 
            (receivers_app['play_id'] == play_data['play_id'])
        ]
        
        input_frames = play_data.get('input_frames', [])
        output_frames = play_data.get('output_frames', [])
        all_frames = sorted(set(input_frames + output_frames))
        pause_frame = play_data['pause_frame']
        
        if phase == 'initial':
            start_frame = input_frames[0] if input_frames else 1
            fig = build_static_figure(frame_data, start_frame, preds, receiver_rankings, 
                                      qb_name=qb_name, los_x=los_x, first_down_x=first_down_x,
                                      show_routes=True, attacking_right=attacking_right)
            
        elif phase == 'running':
            start_frame = input_frames[0] if input_frames else 1
            fig, _ = build_animated_figure(
                frame_data, input_frames, preds, receiver_rankings,
                start_frame=start_frame, end_frame=pause_frame, 
                speed_multiplier=speed, phase='running', qb_name=qb_name,
                los_x=los_x, first_down_x=first_down_x
            )
            
        elif phase == 'paused':
            fig = build_static_figure(frame_data, pause_frame, preds, receiver_rankings, 
                                      show_probs=True, qb_name=qb_name,
                                      los_x=los_x, first_down_x=first_down_x)
            
        elif phase == 'selected':
            fig = build_static_figure(
                frame_data, pause_frame, preds, receiver_rankings,
                show_probs=True, selected_id=selected_id, selection_correct=selection_correct,
                qb_name=qb_name, los_x=los_x, first_down_x=first_down_x
            )
            
        elif phase == 'post-throw':
            fig, _ = build_animated_figure(
                frame_data, all_frames, preds, receiver_rankings,
                start_frame=pause_frame, end_frame=all_frames[-1] if all_frames else pause_frame,
                speed_multiplier=speed, phase='post-throw',
                qb_pos=play_data.get('qb_pos'),
                actual_nfl_id=play_data.get('actual_nfl_id'),
                result=play_data.get('result'),
                qb_name=qb_name, los_x=los_x, first_down_x=first_down_x
            )
            
        elif phase == 'complete':
            end_frame = all_frames[-1] if all_frames else pause_frame
            actual_receiver_id = play_data.get('actual_nfl_id')
            
            ball_pos = None
            if play_data.get('qb_pos') and actual_receiver_id:
                end_players = frame_data.get(end_frame, [])
                for p in end_players:
                    if p['nfl_id'] == actual_receiver_id:
                        ball_pos = [p['x'], p['y']]
                        break
            
            fig = build_static_figure(
                frame_data, end_frame, preds, receiver_rankings,
                show_probs=True, selected_id=selected_id, selection_correct=selection_correct,
                show_ball=True, ball_pos=ball_pos,
                ball_result=play_data.get('result'),
                actual_receiver_id=actual_receiver_id,
                qb_name=qb_name, los_x=los_x, first_down_x=first_down_x
            )
        else:
            fig = create_field_figure()
        
        max_frame = max(all_frames) if all_frames else current_frame
        frame_text = f"Frame {current_frame} / {max_frame}"
        
        return fig, frame_text
    
    @app.callback(
        Output('app-phase', 'data', allow_duplicate=True),
        Output('animation-trigger', 'data'),
        Output('animation-check', 'disabled'),
        Output('animation-check', 'n_intervals'),
        Input('play-btn', 'n_clicks'),
        State('app-phase', 'data'),
        State('play-data', 'data'),
        State('animation-trigger', 'data'),
        prevent_initial_call=True
    )
    def handle_play_button(n_clicks, phase, play_data, trigger):
        if play_data is None:
            return 'initial', trigger, True, 0
        
        if phase == 'initial':
            return 'running', trigger + 1, False, 0
        elif phase == 'paused':
            return 'paused', trigger, True, 0
        elif phase == 'selected':
            return 'post-throw', trigger + 1, False, 0
        elif phase == 'running':
            return 'paused', trigger, True, 0
        elif phase == 'post-throw':
            return 'post-throw', trigger, False, 0
        elif phase == 'complete':
            return 'complete', trigger, True, 0
        
        return phase, trigger, True, 0
    
    @app.callback(
        Output('app-phase', 'data', allow_duplicate=True),
        Output('animation-check', 'disabled', allow_duplicate=True),
        Output('current-frame', 'data', allow_duplicate=True),
        Input('animation-check', 'n_intervals'),
        State('app-phase', 'data'),
        State('play-data', 'data'),
        State('frame-duration', 'data'),
        prevent_initial_call=True
    )
    def check_animation_status(n_intervals, phase, play_data, frame_duration):
        if play_data is None:
            return 'initial', True, 1
        
        input_frames = play_data.get('input_frames', [])
        output_frames = play_data.get('output_frames', [])
        pause_frame = play_data['pause_frame']
        all_frames = sorted(set(input_frames + output_frames))
        
        frames_per_interval = 100 / frame_duration
        
        if phase == 'running':
            running_frames = [f for f in input_frames if f <= pause_frame]
            expected_intervals = int(len(running_frames) / frames_per_interval) + 5
            
            if n_intervals >= expected_intervals:
                return 'paused', True, pause_frame
        
        elif phase == 'post-throw':
            post_frames = [f for f in all_frames if f >= pause_frame]
            expected_intervals = int(len(post_frames) / frames_per_interval) + 5
            
            if n_intervals >= expected_intervals:
                end_frame = all_frames[-1] if all_frames else pause_frame
                return 'complete', True, end_frame
        
        return no_update, no_update, no_update
    
    @app.callback(
        Output('feedback-panel', 'children'),
        Output('feedback-panel', 'style', allow_duplicate=True),
        Output('user-selection', 'data', allow_duplicate=True),
        Output('selection-correct', 'data', allow_duplicate=True),
        Output('app-phase', 'data', allow_duplicate=True),
        Output('ep-display', 'style', allow_duplicate=True),
        Output('timer-interval', 'disabled', allow_duplicate=True),
        Input('field', 'clickData'),
        State('play-data', 'data'),
        State('app-phase', 'data'),
        prevent_initial_call=True
    )
    def handle_selection(click_data, play_data, phase):
        hidden_style = {
            'display': 'none',
            'width': '280px',
            'marginLeft': '20px',
            'alignSelf': 'center'
        }
        
        ep_hidden_style = {'position': 'absolute', 'left': '160px', 'top': '100px', 'backgroundColor': 'rgba(30,30,30,0.9)',
                          'padding': '4px 10px', 'borderRadius': '4px', 'border': '1px solid white', 'zIndex': '100', 'display': 'none'}
        
        if click_data is None or play_data is None:
            return "", hidden_style, None, None, phase, ep_hidden_style, True
        
        if phase != 'paused':
            return "", hidden_style, None, None, phase, no_update, no_update
        
        point = click_data['points'][0]
        if 'customdata' not in point:
            return "", hidden_style, None, None, phase, no_update, no_update
        
        selected_id = point['customdata']
        if isinstance(selected_id, list):
            selected_id = selected_id[0]
        
        preds = receivers_app[
            (receivers_app['game_id'] == play_data['game_id']) & 
            (receivers_app['play_id'] == play_data['play_id'])
        ]
        
        selected_row = preds[preds['nfl_id'] == selected_id]
        if len(selected_row) == 0:
            return "", hidden_style, None, None, phase, no_update, no_update
        
        selected_row = selected_row.iloc[0]
        selected_prob = selected_row['catch_probability']
        selected_route = selected_row.get('predicted_route', '?')
        selected_name = selected_row.get('receiver_name', 'Unknown')
        
        is_best = selected_id == play_data['best_nfl_id']
        
        if is_best:
            your_choice_color = '#4caf50'
        elif selected_prob >= play_data['best_prob'] * 0.8:
            your_choice_color = '#ff9800'
        else:
            your_choice_color = '#f44336'
        
        section_style = {'marginBottom': '6px'}
        label_style = {'color': '#888', 'fontSize': '9px', 'marginBottom': '0px', 'textTransform': 'uppercase', 'margin': '0'}
        name_style = {'color': 'white', 'fontWeight': 'bold', 'fontSize': '12px', 'marginBottom': '0px', 'margin': '0'}
        detail_style = {'color': '#aaa', 'fontSize': '10px', 'margin': '0'}
        
        your_choice = html.Div([
            html.P("Your Choice:", style=label_style),
            html.P(selected_name, style={**name_style, 'color': your_choice_color}),
            html.P(f"{selected_route} | {selected_prob*100:.0f}%", style=detail_style)
        ], style=section_style)
        
        best_choice = html.Div([
            html.P("Best Choice:", style=label_style),
            html.P(play_data.get('best_name', 'Unknown'), style={**name_style, 'color': '#ffd700'}),
            html.P(f"{play_data['best_route']} | {play_data['best_prob']*100:.0f}%", style=detail_style)
        ], style=section_style)
        
        if 'actual_nfl_id' in play_data:
            if play_data['result'] == 'CAUGHT':
                yards = play_data.get('yards_gained', 0)
                is_td = play_data.get('is_touchdown', False)
                if is_td:
                    result_text = f"CAUGHT ✓ | {yards} YDS | TOUCHDOWN! 🏈"
                else:
                    result_text = f"CAUGHT ✓ | {yards} YDS"
                result_color = '#4caf50'
            else:
                result_text = "INCOMPLETE ✗"
                result_color = '#f44336'
            
            qb_choice = html.Div([
                html.P("QB's Choice:", style=label_style),
                html.P(play_data.get('actual_name', 'Unknown'), style=name_style),
                html.P(f"{play_data.get('actual_route', '?')} | {play_data.get('actual_prob', 0)*100:.0f}%", style=detail_style),
                html.P(result_text, style={'color': result_color, 'fontSize': '10px', 'fontWeight': 'bold', 'margin': '0'})
            ], style=section_style)
        else:
            qb_choice = html.Div([
                html.P("QB's Choice:", style=label_style),
                html.P("Unknown", style=name_style)
            ], style=section_style)
        
        feedback = html.Div([
            your_choice,
            html.Hr(style={'borderColor': '#444', 'margin': '3px 0'}),
            best_choice,
            html.Hr(style={'borderColor': '#444', 'margin': '3px 0'}),
            qb_choice,
            html.P("Press Play →", style={'color': '#666', 'marginTop': '4px', 'marginBottom': '0', 'fontSize': '9px', 'textAlign': 'center'})
        ])
        
        show_style = {
            'display': 'block',
            'width': '170px',
            'position': 'absolute',
            'right': '90px',
            'top': '168px',
            'backgroundColor': '#2a2a2a',
            'padding': '10px',
            'borderRadius': '8px',
            'zIndex': '1000'
        }
        
        ep_show_style = {'position': 'absolute', 'left': '160px', 'top': '100px', 'backgroundColor': 'rgba(30,30,30,0.9)',
                         'padding': '4px 10px', 'borderRadius': '4px', 'border': '1px solid white', 'zIndex': '100', 'display': 'block'}
        
        return feedback, show_style, selected_id, is_best, 'selected', ep_show_style, True
    
    # Timer start callback - starts when phase becomes 'paused'
    @app.callback(
        Output('timer-remaining', 'data', allow_duplicate=True),
        Output('timer-interval', 'disabled', allow_duplicate=True),
        Output('timer-display', 'children', allow_duplicate=True),
        Input('app-phase', 'data'),
        State('timer-dropdown', 'value'),
        prevent_initial_call=True
    )
    def start_timer(phase, timer_setting):
        if phase == 'paused' and timer_setting > 0:
            return timer_setting, False, f"{timer_setting}s"
        elif phase == 'paused' and timer_setting == 0:
            return None, True, ""
        else:
            return None, True, ""
    
    # Timer countdown callback
    @app.callback(
        Output('timer-remaining', 'data', allow_duplicate=True),
        Output('timer-display', 'children', allow_duplicate=True),
        Output('timer-interval', 'disabled', allow_duplicate=True),
        Input('timer-interval', 'n_intervals'),
        State('timer-remaining', 'data'),
        State('app-phase', 'data'),
        prevent_initial_call=True
    )
    def countdown_timer(n_intervals, remaining, phase):
        if remaining is None or phase != 'paused':
            return no_update, "", True
        
        new_remaining = remaining - 1
        
        if new_remaining <= 0:
            return 0, "0s", True
        else:
            return new_remaining, f"{new_remaining}s", False
    
    return app

In [4]:
# %% [markdown]
# # Cell 6: Run App

# %%
app = create_app()
app.run(debug=True, port=8050, jupyter_mode='external')

Dash app running on http://127.0.0.1:8050/
